In [1]:
import requests
import pandas as pd 
import json
import pymongo
from pymongo import MongoClient

In [2]:
table_name = 'UnemploymentRate.csv'

In [2]:
df_local_msa = pd.read_excel('../Local_Area/local_area_msa.xlsx')
df_local_msa

,CBSA,MSA_Name,area_code
0,11500,"Anniston-Oxford, AL",MT0111500000000
1,12220,"Auburn-Opelika, AL",MT0112220000000
2,13820,"Birmingham-Hoover, AL",MT0113820000000
3,19300,"Daphne-Fairhope-Foley, AL",MT0119300000000
4,19460,"Decatur, AL",MT0119460000000
...,...,...,...
379,39100,"Poughkeepsie-Newburgh-Middletown, NY",DV3620524000000
380,39150,"Prescott Valley-Prescott, AZ",MT0439140000000
381,39300,"Providence-Warwick, RI-MA",MT4477200000000
382,44140,"Springfield, MA",MT2578100000000


In [23]:
df = pd.DataFrame(columns=["series id","year","period","period_name","value"])
series = []
for index, row in df_local_msa.iterrows():
    # construct series id by concatenating: Prefix + MSA Area Code + Measure Code
    series.append('LAU' + row['area_code'] + '03')

In [24]:
series[0]

'LAUMT011150000000003'

In [25]:
def request_series(series_list):
    #  print(series_list)
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": series_list,"startyear":"2010", "endyear":"2019","registrationkey":"<key here>"})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    if json_data['status'] == 'REQUEST_SUCCEEDED':
        add_to_df(json_data)
    else:
        print(json_data['status'])

In [26]:
def add_to_df(data):
    for series in data['Results']['series']:
        seriesId = series['seriesID']
        for item in series['data']:
            year = item['year']
            period = item['period']
            period_name = item["periodName"]
            value = item['value']
            if 'M01' <= period <= 'M12':
                df.loc[len(df)] = [seriesId,year,period,period_name,value]

In [27]:
lister = [series[i:i + 50] for i in range(0, len(series), 50)]

for i in range(0, len(lister)):
    request_series(lister[i])
    print(i)

0
1
2
3
4
5
6
7


In [28]:
df

,series id,year,period,period_name,value
0,LAUMT011150000000003,2019,M12,December,3.0
1,LAUMT011150000000003,2019,M11,November,2.9
2,LAUMT011150000000003,2019,M10,October,3.0
3,LAUMT011150000000003,2019,M09,September,3.0
4,LAUMT011150000000003,2019,M08,August,3.4
...,...,...,...,...,...
46075,LAUMT257960000000003,2010,M05,May,8.8
46076,LAUMT257960000000003,2010,M04,April,8.8
46077,LAUMT257960000000003,2010,M03,March,9.4
46078,LAUMT257960000000003,2010,M02,February,9.6


In [38]:
len(df['series id'].unique())

384

In [29]:
df.groupby('series id').count()

,year,period,period_name,value
series id,,,,
LAUDV362052400000003,120,120,120,120
LAUMT011150000000003,120,120,120,120
LAUMT011222000000003,120,120,120,120
LAUMT011382000000003,120,120,120,120
LAUMT011930000000003,120,120,120,120
...,...,...,...,...
LAUMT553954000000003,120,120,120,120
LAUMT554310000000003,120,120,120,120
LAUMT554814000000003,120,120,120,120


In [5]:
df_cbsa = df_local_msa[['CBSA', 'area_code']].copy()
df_cbsa.head()

,CBSA,area_code
0,11500,MT0111500000000
1,12220,MT0112220000000
2,13820,MT0113820000000
3,19300,MT0119300000000
4,19460,MT0119460000000


In [6]:
df_cbsa['series id'] = 'LAU' + df_cbsa['area_code'] + '03'
df_cbsa

,CBSA,area_code,series id
0,11500,MT0111500000000,LAUMT011150000000003
1,12220,MT0112220000000,LAUMT011222000000003
2,13820,MT0113820000000,LAUMT011382000000003
3,19300,MT0119300000000,LAUMT011930000000003
4,19460,MT0119460000000,LAUMT011946000000003
...,...,...,...
379,39100,DV3620524000000,LAUDV362052400000003
380,39150,MT0439140000000,LAUMT043914000000003
381,39300,MT4477200000000,LAUMT447720000000003
382,44140,MT2578100000000,LAUMT257810000000003


In [7]:
df_combined = pd.merge(df_cbsa, df, on="series id")
df_combined

,CBSA,area_code,series id,year,period,period_name,value
0,11500,MT0111500000000,LAUMT011150000000003,2019,M12,December,3.0
1,11500,MT0111500000000,LAUMT011150000000003,2019,M11,November,2.9
2,11500,MT0111500000000,LAUMT011150000000003,2019,M10,October,3.0
3,11500,MT0111500000000,LAUMT011150000000003,2019,M09,September,3.0
4,11500,MT0111500000000,LAUMT011150000000003,2019,M08,August,3.4
...,...,...,...,...,...,...,...
46075,49340,MT2579600000000,LAUMT257960000000003,2010,M05,May,8.8
46076,49340,MT2579600000000,LAUMT257960000000003,2010,M04,April,8.8
46077,49340,MT2579600000000,LAUMT257960000000003,2010,M03,March,9.4
46078,49340,MT2579600000000,LAUMT257960000000003,2010,M02,February,9.6


In [8]:
df_combined.drop(columns=['area_code'], inplace=True)
df_combined.head()

,CBSA,series id,year,period,period_name,value
0,11500,LAUMT011150000000003,2019,M12,December,3.0
1,11500,LAUMT011150000000003,2019,M11,November,2.9
2,11500,LAUMT011150000000003,2019,M10,October,3.0
3,11500,LAUMT011150000000003,2019,M09,September,3.0
4,11500,LAUMT011150000000003,2019,M08,August,3.4


In [9]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46080 entries, 0 to 46079
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CBSA         46080 non-null  int64  
 1   series id    46080 non-null  object 
 2   year         46080 non-null  int64  
 3   period       46080 non-null  object 
 4   period_name  46080 non-null  object 
 5   value        46080 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 3.7+ MB


In [43]:
df.to_csv('../CSVs/UnemploymentRate.csv', index=False)

In [3]:
df = pd.read_csv('../CSVs/UnemploymentRate.csv')
df

,series id,year,period,period_name,value
0,LAUMT011150000000003,2019,M12,December,3.0
1,LAUMT011150000000003,2019,M11,November,2.9
2,LAUMT011150000000003,2019,M10,October,3.0
3,LAUMT011150000000003,2019,M09,September,3.0
4,LAUMT011150000000003,2019,M08,August,3.4
...,...,...,...,...,...
46075,LAUMT257960000000003,2010,M05,May,8.8
46076,LAUMT257960000000003,2010,M04,April,8.8
46077,LAUMT257960000000003,2010,M03,March,9.4
46078,LAUMT257960000000003,2010,M02,February,9.6


In [10]:
# Create instance of MongoClient
client = MongoClient()
# Connection URI
client = MongoClient('<conn string here>')
# Select database
db = client['MSA']
# create new collection
collection = db.UnemploymentRate_raw
# turn dataframe into readable format for mongo
df_dict = df_combined.to_dict(orient='records')
# write dataframe to unemployment_predicted_2024 collection
collection.insert_many(df_dict)